<a href="https://colab.research.google.com/github/GasKitela/ia-utn/blob/main/Entrega2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile

uploaded = files.upload()

data = zipfile.ZipFile(io.BytesIO(uploaded['Evoluciones_Agumon.zip']), 'r')
data.extractall()


In [ ]:
from google.colab import files
import zipfile
import io
import cv2 #reconocimiento de imagenes en py
import numpy as np #arrays
import os
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
import keras.optimizers as Optimizer

try: 
  %tensorflow_version 2.x
except Exception: 
  pass

ruta_agumon="/content/Agumon"
ruta_greymon="/content/Greymon"
ruta_metalgreymon="/content/Metalgreymon"
ruta_wargreymon="/content/Wargreymon"

agumon=[]
greymon=[]
metalgreymon=[]
wargreymon=[]
img_size=150

for img in os.listdir(ruta_agumon):
  if('jpg' in img):
    img = cv2.imread(os.path.join(ruta_agumon, img))
    img_resize = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
    agumon.append(img_resize)

print("Elementos agumon", len(agumon))

for img in os.listdir(ruta_greymon):
  if('jpg' in img):
    img = cv2.imread(os.path.join(ruta_greymon, img))
    img_resize = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
    greymon.append(img_resize)

print("Elementos greymon", len(greymon))

for img in os.listdir(ruta_metalgreymon):
  if('jpg' in img):
    img = cv2.imread(os.path.join(ruta_metalgreymon, img))
    img_resize = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
    metalgreymon.append(img_resize)

print("Elementos metalgreymon", len(metalgreymon))

for img in os.listdir(ruta_wargreymon):
  if('jpg' in img):
    img = cv2.imread(os.path.join(ruta_wargreymon, img))
    img_resize = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
    wargreymon.append(img_resize)

print("Elementos wargreymon", len(wargreymon))

Agumon = np.array(agumon)
Greymon = np.array(greymon)
Metalgreymon = np.array(metalgreymon)
Wargreymon = np.array(wargreymon)

#juntar imagenes
imagenes2 = np.concatenate([Agumon, Greymon])
imagenes1 = np.concatenate([imagenes2, Metalgreymon])
imagenes = np.concatenate([imagenes1, Wargreymon])

Imagenes = np.array(imagenes)

etiquetas_agumon = np.repeat(0,21)
etiquetas_greymon = np.repeat(1,26)
etiquetas_metalgreymon = np.repeat(2,21)
etiquetas_wargreymon = np.repeat(3,26)

labels2 = np.concatenate([etiquetas_agumon, etiquetas_greymon])
labels1 = np.concatenate([labels2, etiquetas_metalgreymon])
labels = np.concatenate([labels1, etiquetas_wargreymon])

Labels = np.array(labels)

clases = ['Agumon', 'Greymon', 'Metalgreymon', 'Wargreymon']

plt.figure(figsize=(10,10))

for i in range(94):
  plt.subplot(10,10,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  img = Imagenes[i]
  plt.imshow(img)
  plt.xlabel(clases[Labels[i]])

plt.show()

## hasta aca etiquetas

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(150,150,3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
trained = model.fit(Imagenes, Labels, epochs=92)

print("Cargar imagen prediccion.jpg para clasificar:")
uploaded = files.upload()

img = cv2.imread("/content/prediccion.jpg")
img_resize = cv2.resize(img, (img_size,img_size), interpolation=cv2.INTER_AREA)
imagen_pred = np.expand_dims(img_resize, 0)

prediction = model.predict(imagen_pred)

threshold = 0.5  # Umbral mínimo de probabilidad

max_probability = np.max(prediction)
predicted_class = np.argmax(prediction)

if max_probability < threshold:
    print("La imagen no se ajusta a ninguna clasificación.")
else:
    print("La clasificación predecida es", clases[predicted_class])
    

